In [ ]:
%config Completer.use_jedi=False

import sys
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import rankdata
import scipy.optimize
sys.path.append('../../')

def get_line_type(fname):
    if (fname in ['microbes','monomers','substrates','degradation_enzymes']):
        res = '-'
    else:
        res = '--'
    return res
from refsimu import reload_simulation
from refsimu import microbe_pop_on

In [ ]:
def linreg_2d(x1,x2,y):
    A = np.zeros((x1.shape[0],3))
    A[:,0] = 1
    A[:,1] = x1
    A[:,2] = x2
    res = scipy.optimize.nnls(A,y)
    return res

In [ ]:
N = 64
x1 = np.random.rand(N)
x2 = np.random.rand(N)
y = 3 + 2.0*x1 + 5.0 * x2
machin = linreg_2d(x1,x2,y)
print(machin)

In [ ]:
microbe_pop_on()

In [ ]:
# base_dir = './BIDULETESTMEM/' # CHANGE FOR YOUR OWN 
base_dir = './TRUC/'
sdirs = [f for f in glob.glob(os.path.join(base_dir,'*')) if os.path.isdir(f)]
dlists = {}
for sdir in sdirs:
#     print(sdir)
    skey = '_'.join(os.path.basename(sdir).split('_')[1:3])
#     print(skey)
    dlist = glob.glob(os.path.join(sdir,'replica_*'))
    dlist = [d for d in dlist if os.path.isdir(d)]
    dlists[skey] = dlist

# dictionnary dlists is keyed by names of the base_dir subdirectories ( CELLULOSE conditions), the values are lists
# of the subdirectories containing the various replica
for k,d in dlists.items():
    print('*'*20,'\n',k)
    for i,r in enumerate(d):
        print(i,r)

In [ ]:
rep_dir = dlists['GRASSLAND_NoMut'][0] # select condition and replica 
print('Reloading from {}'.format(rep_dir))
eco = reload_simulation(rep_dir,-1)
# print(eco.grid_shape)

diagcollector = eco.get_diag_collector()
tl_files = sorted(glob.glob('{}/timelines_dump_*.hdf5'.format(rep_dir)))
# print(tl_files)
dump_file = tl_files[0]
diagcollector.load_from_dumpfile(dump_file)
tla_comp_space = diagcollector.get_timeline_arrays('space_comp_sum', tag_filter_func= lambda t: 'pool' in t and 'input' not in t)
tla_comp_space_external =  diagcollector.get_timeline_arrays('space_comp_sum', tag_filter_func= lambda t: 'pool' in t and 'input'  in t)
tla_recycling = diagcollector.get_timeline_arrays('space_comp_sum',tag_filter_func=lambda t:'stoechio_balance_recycler' in t)
tla_mortality = diagcollector.get_timeline_arrays('space_comp_sum',tag_filter_func=lambda t:'mortality_op' in t)
tla_bytype = diagcollector.get_timeline_arrays('sum_by_type')
tla_ncells = diagcollector.get_timeline_arrays('ncells')
tla_sub = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'substrates' in t)
tla_enz = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'degradation_enzymes' in t)
tla_mon = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'monomers' in t)
tla_mic = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'microbes' in t)
tla_mic_ntax = diagcollector.get_timeline_arrays('ntaxpercell')
tla_clim = diagcollector.get_timeline_arrays('Climate')
tla_mic_nind = diagcollector.get_timeline_arrays('Quanta')
tla_mic_indmass = diagcollector.get_timeline_arrays('LocalTaxIndividualMass')
tla_micclasspop = diagcollector.get_timeline_arrays('TaxClassesPop')
tla_subclassmass = diagcollector.get_timeline_arrays('SubClassesTMass')
tla_enzclassmass = diagcollector.get_timeline_arrays('EnzClassesTMass')

In [ ]:
tmin = 0
tmax = -1
# tmin = 365*20
# tmax = tmin+(6) * 365

In [ ]:
tla_mem = diagcollector.get_timeline_arrays('Memory')
print(tla_mem.keys())
for k,d in tla_mem.items():
    fig,ax = plt.subplots(1,1,figsize=(12,3))
    ax.plot(d['times'],d['values'] / (2**20))
    ax.grid()
    ax.set_ylabel('Mo')
    ax.set_title(k)
# fig,ax = plt.subplots(1,1,figsize=(12,3))
# ax.plot(tla_mem['ru_maxrss']['times'],tla_mem['ru_maxrss']['values'])
# ax.grid()

In [ ]:
def get_yearly_averages(times_day,data):
    period = 365
    tyear_full = times_day.astype(int) // 365
    tyear_comp = np.sort(np.unique(tyear_full))
    data_comp_shap = tyear_comp.shape + data.shape[1:]
    data_comp = np.zeros(data_comp_shap,dtype=data.dtype)
    for iy,year in enumerate(tyear_comp):
        filt = tyear_full == year
        data_comp[iy,...] = np.mean(data[filt,...],axis=0)
    return tyear_comp,data_comp

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
for k,d in tla_micclasspop.items():
    ax.plot(d['times'],d['values'],label=k)
ax.set_xlabel('time (days)')
ax.set_ylabel('Population')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();


In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
for k,d in tla_micclasspop.items():
    tt,dd = get_yearly_averages(d['times'],d['values'])
    ax.plot(tt,dd,'-o',label=k)
ax.set_xlabel('time (years)')
ax.set_ylabel('Population (yearly average)')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
tt = [d['times'] for d in tla_subclassmass.values()]
vv = np.row_stack([d['values'][:,0] for d in tla_subclassmass.values()])
kk = list(tla_subclassmass.keys())
ax.stackplot(tt[0],vv,labels=kk)
# for k,d in tla_subclassmass.items():
#     ax.plot(d['times'],d['values'],label=k)
ax.set_xlabel('time (days)')
ax.set_ylabel('Total mass')
ax.set_title('Litter Composition')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
ttvv = [get_yearly_averages(d['times'],d['values'][:,0]) for d in tla_subclassmass.values()]
vv = np.row_stack([d[1] for d in ttvv])
kk = list(tla_subclassmass.keys())
ax.stackplot(ttvv[0][0],vv,labels=kk)
# for k,d in tla_subclassmass.items():
#     ax.plot(d['times'],d['values'],label=k)
ax.set_xlabel('time (year)')
ax.set_ylabel('Total mass (yearly average)')
ax.set_title('Litter Composition')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
for k,d in tla_subclassmass.items():
    ax.plot(d['times'],d['values'],label=k)
ax.set_xlabel('time (days)')
ax.set_ylabel('Total mass')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
for k,d in tla_subclassmass.items():
    tt,dd = get_yearly_averages(d['times'],d['values'])
    ax.plot(tt,dd,label=k)
ax.set_xlabel('time (year)')
ax.set_ylabel('Total mass (yearly average)')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

In [ ]:
def get_neg_logdecay(v):
    pfilt = v > 0
    lgv = np.zeros_like(v)
    lgv[pfilt] = np.log(v[pfilt])
    dlgv = np.gradient(lgv)
    dlgv[dlgv > 0] = 0
    return -dlgv

fig,ax = plt.subplots(1,1, figsize=(12,5))
for k,d in tla_subclassmass.items():
    ax.plot(d['times'],get_neg_logdecay(d['values'][:,0]),label=k)
ax.set_xlabel('time (days)')
ax.set_ylabel('Total mass decay rate')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

$$
d_t S = -V_m \frac{S}{K_m +S}
$$
$$
d_t \ln(S) = -\frac{V_m}{{K_m +S}}
$$

$$
d_t \ln(S)(K_m + S) = -V_m
$$
$$
S d_t \ln(S) = -V_m(E(t))  - K_m d_t \ln(S)
$$

$$
\frac{
S d_t \ln(S)
}{E}= -\frac{V_m(E(t))}{E}  - K_m \frac{d_t \ln(S)}{E}
$$
$$
Y = \frac{d_t \ln(S)}{E}
$$

$$
\frac{V_m}{E} \approx V_0
$$
$$
S * Y = -V_0 - K_m Y
$$

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
for k,d in tla_subclassmass.items():
    tt,dd = get_yearly_averages(d['times'],d['values'][:,0])
    ax.plot(tt,get_neg_logdecay(dd),label=k)
ax.set_xlabel('time (year)')
ax.set_ylabel('Total mass decay rate')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

In [ ]:
fix,axes = plt.subplots(1,3,figsize=(14,4))
for (k,d),ax in zip(tla_subclassmass.items(),axes):
    S = d['values'][tmin:tmax,0]
    mdolgS = get_neg_logdecay(S)
    ax.scatter(mdolgS,mdolgS *S,c = d['times'][tmin:tmax],s=0.1)
    ax.set_xlabel(r'$-d_t \ln(S)$')
    ax.set_ylabel(r'$-d_t \ln(S)*S$')
    ax.set_title(k)


In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))
for k,d in tla_enzclassmass.items():
    ax.plot(d['times'],d['values'],label=k)
ax.set_xlabel('time (days)')
ax.set_ylabel('Enzyme Total mass')
ymin,ymax = ax.get_ylim()
ax.set_ylim(0,ymax)
ax.legend();
ax.grid();

In [ ]:
fix,axes = plt.subplots(1,3,figsize=(14,4))
tmin = 2
tmax =-1
for (k,d),ax in zip(tla_subclassmass.items(),axes):
    S = d['values'][tmin:tmax,0]
    E = tla_enzclassmass[k]['values'][tmin:tmax,0]
    Y = get_neg_logdecay(S) / E
    ax.scatter(Y,Y*S,c = d['times'][tmin:tmax],s = 0.1)
    ax.set_xlabel(r'$(-d_t \ln(S)/E)$')
    ax.set_ylabel(r'$(-d_t \ln(S)/E)*S$')
    ax.set_title(k)
    ax.grid()

In [ ]:
f = eco.functions_module.taxon_selection_map
tax_classes = {k : np.array([i for i in range(eco.microbes.n_taxa) if f(i)[0]== k])
               for k in ['MICr','MICk']}
tax_Cmass_by_class = {}
for kclass, tax_sel in tax_classes.items():
    tax_Cmass_by_class[kclass] = np.sum(tla_mic['microbes']['values'][:,:,0][:,tax_sel],axis=-1)
tax_Cmass_by_class['MICr+MICk'] = tax_Cmass_by_class['MICr'] + tax_Cmass_by_class['MICk']

In [ ]:
ne = len(tla_enzclassmass)
nt = len(tax_Cmass_by_class)
itmin = 2000
itmax = -1
# print(ne,nt)
fig, axes = plt.subplots(ne,nt,figsize=(4*nt,3*ne))
for ie,(ke,de) in enumerate(tla_enzclassmass.items()):
    for it,(kt,dt) in enumerate(tax_Cmass_by_class.items()):
#         print(it,kt)
#         print(dt['values'].shape)
        axes[ie,it].scatter(dt[itmin:itmax],
                            de['values'][itmin:itmax],
                            c=de['times'][itmin:itmax],s=0.5)
#         print(de['values'].shape)
#         print(dt.shape)
        
        tmp_reg  = scipy.stats.linregress(dt[itmin:itmax],de['values'][itmin:itmax,0])
#         print(tmp_reg)
        prev = tmp_reg.intercept + tmp_reg.slope * dt[itmin:itmax]
        axes[ie,it].plot(dt[itmin:itmax],prev,'--',label='slope {:e}'.format(tmp_reg.slope))
        axes[ie,it].set_xlabel('Cmass {}'.format(kt))
        axes[ie,it].set_ylabel('Enz {}'.format(ke))
        axes[ie,it].grid()
        axes[ie,it].legend()
fig.tight_layout()
plt.show()
plt.close(fig)

In [ ]:
regs_2d = {}
for ie,(ke,de) in enumerate(tla_enzclassmass.items()):
    print(ie,ke)
    y = de['values'][:,0]
    x1 = tax_Cmass_by_class['MICr'][:]
    x2 = tax_Cmass_by_class['MICk'][:]
    res = linreg_2d(x1,x2,y)
    print(res)
    coeffs = res[0]
    pred = coeffs[0] + coeffs[1] * x1 + coeffs[2] * x2
    regs_2d[ke] = pred

In [ ]:
print(regs_2d.keys())

In [ ]:
fix,axes = plt.subplots(1,3,figsize=(14,4))
tmin = 1000
tmax =-1
for (k,d),ax in zip(tla_subclassmass.items(),axes):
    S = d['values'][tmin:tmax,0]
#     E = tla_enzclassmass[k]['values'][tmin:tmax,0]
    E = regs_2d[k][tmin:tmax]
    Y = get_neg_logdecay(S) / E
    ax.scatter(Y,Y*S,c = d['times'][tmin:tmax],s = 0.1)
    tmp_reg  = scipy.stats.linregress(Y[tmin:tmax],(Y*S)[tmin:tmax])
    ymin,ymax = ax.get_ylim()
    xmin,xmax = ax.get_xlim()
    reg_pred = tmp_reg.intercept  + tmp_reg.slope * Y[tmin:tmax]
    ax.plot(Y[tmin:tmax],reg_pred,'--',lw=0.5)
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.set_xlabel(r'$(-d_t \ln(S)/E)$')
    ax.set_ylabel(r'$(-d_t \ln(S)/E)*S$')
    ax.set_title(k)
    ax.grid()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
for itax in range(tla_mic_nind['microbes']['values'].shape[1]):
    ax.plot(tla_mic_nind['microbes']['times'],tla_mic_nind['microbes']['values'][:,itax],label='Taxon {}'.format(itax)
           )
# ax.legend()
ax.grid()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
tt,dd = get_yearly_averages(tla_mic_nind['microbes']['times'], tla_mic_nind['microbes']['values'])
for itax in range(tla_mic_nind['microbes']['values'].shape[1]):
#     ax.plot(tla_mic_nind['microbes']['times'],tla_mic_nind['microbes']['values'][:,itax],label='Taxon {}'.format(itax)
#            )
    ax.plot(tt,dd[:,itax],label='Taxon {}'.format(itax)
           )
# ax.legend()
ax.grid()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
ax.plot(tla_mic_nind['microbes']['times'],np.sum(tla_mic_nind['microbes']['values'],axis=(-2,-1)))
ax.set_xlabel('time')
ax.set_title('Nb individuals')
ax.grid();

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(12,4))
for k, ax in zip(tla_clim.keys(),axes):
    ax.plot(tla_clim[k]['times'],tla_clim[k]['values'])
    ax.set_xlabel('time')
    ax.set_ylabel(k)
    ax.grid()

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    tmp_sum = None
    for k, d in tla_comp_space.items():
        if (tmp_sum is None):
            tmp_sum = np.zeros_like(d['values'][:,ia])
            td = d['times']
        ls = get_line_type(k)
        ax.plot(d['times'],scal*d['values'][:,ia],ls, label=k)
        tmp_sum += d['values'][:, ia]
    for k, d in tla_comp_space_external.items():
        if (tmp_sum is None):
            tmp_sum = np.zeros_like(d['values'][:,ia])
            td = d['times']
        ls = get_line_type(k)
        ax.plot(d['times'],scal*d['values'][:,ia],ls, label=k)
        tmp_sum -= d['values'][:, ia]
    ax.plot(td,scal*tmp_sum,'.',color='black',lw=0.2,alpha=0.1,label='sum internal - sum sources')
    ax.grid()
    ax.legend(fontsize=8, ncol=3,loc=1, framealpha=0.5)
    ax.set_title('Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
scal = 1.0 / eco.microbes.grid_size
d = tla_comp_space['Respiration_Growth_Induci_Maint']
ax.plot(d['times'],d['values'][:,0]*scal)

ax.grid()
ax.set_title('Respiration Growth Induci Maint')

In [ ]:
fig, ax = plt.subplots(1,1)
t, dat = tla_mic_ntax['microbes']['times'],tla_mic_ntax['microbes']['values']
ax.plot(t,dat[:,0], label='mean')
ax.fill_between(t,dat[:,0]-dat[:,1],dat[:,0]+dat[:,1], alpha=0.2)
ax.plot(t,dat[:,2], label='min')
ax.plot(t,dat[:,3], label='max')
ax.grid()
ax.set_xlabel('Time')
ax.set_ylabel('Live Taxa per grid cell')
ax.legend();
plt.show()
plt.close(fig)
#

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_sub.items():
        for isub, subname in enumerate(eco.substrates.names):
            if (isub < 2):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'][tmin:tmax],scal*d['values'][:,isub,ia][tmin:tmax],ls,label=subname)
    ax.legend()
    ax.grid()
    ax.set_title('Substrate Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
# tmin = 30280
# tmax = tmin + 730
# fig, ax = plt.subplots(1,1, figsize=(16,5))
# for k,d in tla_sub.items():
#     for isub, subname in enumerate(eco.substrates.names):
#         if (isub < 2):
#             ls = '--'
#         else:
#             ls = '-'
#         ssub = np.sum(d['values'][:,isub,:],axis=-1)
#         drate = np.gradient(np.log(ssub+1.0e-16))
#         drate_neg = np.zeros_like(drate)
#         drate_neg[drate < 0]= -drate[drate < 0]
#         ax.plot(d['times'][tmin:tmax],drate_neg[tmin:tmax],ls,label=subname)
# ax.legend()
# ax.grid()
# ax.set_title('Substrate Negative decay rate')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,5))
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_sub.items():
    ax.plot(d['times'][tmin:tmax],scal*np.sum(d['values'][tmin:tmax],axis=(-2,-1)))
ax.grid()
ax.set_title('Substrate total mass')

In [ ]:
# fig, axes = plt.subplots(3,1, figsize=(16,18))
# scal = 1.0 / eco.microbes.grid_size
# tmin = 0
# tmax = -1
# for ia,ax in enumerate(axes):
#     for k,d in tla_sub.items():

#         for isub, subname in enumerate(eco.substrates.names):
#             if (subname not in ['DeadMic','DeadEnz']):
#                 continue
#             if (isub < 2):
#                 ls = '--'
#             else:
#                 ls = '-'
#             ax.plot(d['times'][tmin:tmax],scal*d['values'][:,isub,ia][tmin:tmax],ls,label=subname)
#     ax.legend()
#     ax.grid()
#     ax.set_title('Substrate Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(16,12))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_enz.items():
        for ienz, enzname in enumerate(eco.degradation_enzymes.names):
            if (ienz < 2):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'][tmin:tmax],scal*d['values'][:,ienz,ia][tmin:tmax],ls,label=enzname)
    ax.legend()
    ax.grid()
    ax.set_title('Enzyme Biomass {}'.format(['C','N','P'][ia]))
    ax.set_ylim(0,0.25)

In [ ]:
scal = 1.0 / eco.microbes.grid_size
for k, d in tla_bytype.items():
    mtypes = eco.microbes.get_taxa_types()
    print(mtypes)
    fig, axes = plt.subplots(3,1, figsize=(14,12))
    for ia, ax in enumerate(axes):
        ax.set_title('{} {}'.format(k, ['C','N','P'][ia]))
        for itype, typ in enumerate(mtypes):
            ax.plot(d['times'][tmin:tmax],scal*d['values'][:,itype,ia][tmin:tmax],label=typ)
        ax.grid()
        ax.legend(fontsize=9)

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
dat = np.sum(tla_mic['microbes']['values'][:,:,:],axis=-1)
sdat = np.sum(dat,axis=1)
dat = dat / sdat[:,np.newaxis]
plt.imshow(dat.T, aspect='auto',origin='lower',interpolation='None', cmap='magma')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon Total mass Relative abundancy')
plt.colorbar();

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
ntax = tla_mic['microbes']['values'].shape[1]
dat = np.sum(tla_mic['microbes']['values'][:,:,:],axis=-1)
ranks = rankdata(-dat,method='max', axis=1)
plt.imshow(-ranks.T, aspect='auto',origin='lower',interpolation='None', cmap='hot')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon Total mass rank')
plt.colorbar();

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
dat = np.sum(tla_mic_nind['microbes']['values'][:,:,:],axis=-1)
sdat = np.sum(dat,axis=1)
dat = dat / sdat[:,np.newaxis]
plt.imshow(dat.T, aspect='auto',origin='lower',interpolation='None', cmap='magma')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon Relative pop (individuals)')
plt.colorbar();

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
dat = np.sum(tla_mic_nind['microbes']['values'][:,:,:],axis=-1)
plt.imshow(dat.T, aspect='auto',origin='lower',interpolation='None', cmap='magma')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon pop (individuals)')
plt.colorbar();